# Importation du modèle dans le notebook et analyse de celui-ci 

In [1]:
import joblib
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score, f1_score,classification_report
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from joblib import dump

In [2]:
""" Import de la pipeline utilisée dans l'API et du jeu de données contenant les commentaites. Affichage de la
pipeline pour étudier son contenu"""

modele=joblib.load("sentiment_pipe.joblib")
df=pd.read_csv("comments_train.csv")
modele

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, gamma=0.001))])

In [3]:
""" Affichage des occurences de la colonne sentiment indiquant les classements des commentaires. Cet affichage 
permet de voir que le dataset est déséquilibré."""

X=df["comment"]
y=df["sentiment"]
df["sentiment"].value_counts()

Positive    1019
Negative     598
Name: sentiment, dtype: int64

In [4]:
""" Ici, on place dans deux variables X et y les colonnes du dataset. On remplace les valeurs de la colonne 
sentiment par des valeurs numériques pour permette une interprétation plus facile des résultats. On affiche 
ensuite les indicateurs de performance retenus pour l'étude des prédictions de la pipeline, en prenant en
compte que le dataset est déséquilibré : l'AUC et les métriques recall, precision et f1-score. """



X=df["comment"]
y_pred=modele.predict(X)
y=df["sentiment"].replace({"Positive":1,"Negative":0}).values
resume=classification_report(y_pred,y)
roc_score=roc_auc_score(y_pred,y)
print("AUC : {}".format(roc_score))
print(" ")
print(resume)


AUC : 0.7220142706749327
 
              precision    recall  f1-score   support

           0       0.26      0.76      0.39       206
           1       0.95      0.69      0.80      1411

    accuracy                           0.70      1617
   macro avg       0.61      0.72      0.59      1617
weighted avg       0.86      0.70      0.75      1617



In [5]:
"""Réentrainement du modèle à partir du dataset. On constate une amélioration nette des résultats, notamment sur les
prédictions négatifs (marquées en 0) et l'AUC."""


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
modele.fit(X_train,y_train)
y_pred=modele.predict(X_test)
print(classification_report(y_test, y_pred,zero_division=1))
y_pred2=modele.decision_function(X_test)
roc_score=roc_auc_score(y_test,y_pred2)
print("AUC : ",roc_score)

              precision    recall  f1-score   support

           0       0.85      0.74      0.79       202
           1       0.85      0.92      0.89       332

    accuracy                           0.85       534
   macro avg       0.85      0.83      0.84       534
weighted avg       0.85      0.85      0.85       534

AUC :  0.9279046880591674


In [6]:
# Afin de permettre l'amélioration de la pipeline, on affiche ses paramètres. 

modele.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer()), ('clf', SVC(C=1000, gamma=0.001))],
 'verbose': False,
 'tfidf': TfidfVectorizer(),
 'clf': SVC(C=1000, gamma=0.001),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'clf__C': 1000,
 'clf__break_ties': False,
 'clf__cache_size': 200,
 'clf__class_weight': None,
 'clf__coef0': 0.0,
 'clf__decision_function_shape': 'ovr',
 'clf__degree': 3,
 'clf__gamma': 0.001,
 'clf__kernel': 'rbf',

In [7]:
"""Amélioration du modèle par recherche des paramètres optimales du modèle svm en utilisant une Gridsearch pour
tester différentes combinaisons. Après plusieurs tests, on obtient ici le meilleur résultat que j'ai pu obtenir.
Et le constat est le suivant : l'entraînement du modèle est plus efficace que l'usage d'une GridSearch. Celle-ci 
n'améliore pas les performances de manière significative par rapport à l'entraînement."""


param_svc={
    "clf__kernel": ["sigmoid","rbf","poly"],
    "clf__C" : [1000,10e5,10],
    "clf__gamma" : [0.001,10e-3,0.1],
    "clf__random_state":[42]}
clf=GridSearchCV(modele,param_svc,n_jobs=1,verbose=3)
clf.fit(X_train,y_train)
param_optimales=clf.best_params_
print(param_optimales)

print("Voici les résultats obtenus pour les métriques : ")
y_pred=modele.predict(X_test)
print(classification_report(y_test, y_pred,zero_division=1))
y_pred2=clf.decision_function(X_test)
roc_score=roc_auc_score(y_test,y_pred2)
print("AUC : ",roc_score)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.2s
[CV 2/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.2s
[CV 3/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.2s
[CV 4/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.2s
[CV 5/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.2s
[CV 1/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.2s
[CV 2/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.3s
[CV 3/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.2s
[CV 4/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.3s


In [8]:
print(modele)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, gamma=0.001))])


In [9]:
"""Création d'une nouvelle pipeline reprenant la pipeline précédente avec des paramètres optimisées et
vérification des résultats obtenus. Ils concordent avec ceux attendus."""



modele2 = Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, kernel="sigmoid",gamma=0.001,random_state=42))])
modele2.fit(X_train,y_train)
y_pred=modele2.predict(X_test)
print(classification_report(y_test, y_pred,zero_division=1))
y_pred2=modele2.decision_function(X_test)
roc_score=roc_auc_score(y_test,y_pred2)
print("AUC : ",roc_score)

              precision    recall  f1-score   support

           0       0.84      0.75      0.80       202
           1       0.86      0.92      0.89       332

    accuracy                           0.85       534
   macro avg       0.85      0.83      0.84       534
weighted avg       0.85      0.85      0.85       534

AUC :  0.930320291065251


In [10]:
# sauvegarde de la nouvelle pipeline dans un nouveau fichier joblib

dump(modele2, 'pipeline_optimale.joblib') 

['pipeline_optimale.joblib']